In [212]:
import pandas as pd
import basedosdados as bd
import plotly.express as px

In [213]:
query = """
SELECT candidatos.ano, candidatos.sigla_partido, candidatos.cargo, candidatos.sequencial, candidatos.nome_urna, candidatos.genero, candidatos.raca, resultado_candidatos.resultado, SUM(resultado_candidatos.votos) n_votos
FROM basedosdados.br_tse_eleicoes.candidatos as candidatos
INNER JOIN basedosdados.br_tse_eleicoes.resultados_candidato_municipio as resultado_candidatos
ON (candidatos.sequencial = resultado_candidatos.sequencial_candidato)
AND (candidatos.ano = resultado_candidatos.ano)
WHERE candidatos.sigla_uf = 'RJ'
AND candidatos.ano in (2022, 2018, 2014, 2010)
AND candidatos.cargo = 'deputado federal'
GROUP BY candidatos.ano, candidatos.sigla_uf, candidatos.sequencial, candidatos.sigla_partido, candidatos.cargo, resultado_candidatos.resultado, candidatos.genero, candidatos.raca, candidatos.nome_urna
"""

eleicoes = bd.read_sql(query, billing_project_id="diamundialdainfancia")

Downloading: 100%|██████████| 3846/3846 [00:00<00:00, 4632.80rows/s]


In [214]:
eleicoes = eleicoes.drop_duplicates()

In [215]:
flt = eleicoes['ano'] == 2022
eleicoes_2022 = eleicoes[flt]
eleicoes_2022['% votos'] = ((eleicoes_2022['n_votos'] / eleicoes_2022['n_votos'].sum()) * 100).astype(float).round(2)
eleicoes_2022['nome_partido'] = eleicoes_2022['nome_urna'] + ' (' + eleicoes_2022['sigla_partido'] + ')'

fig = px.bar(eleicoes_2022.sort_values('% votos', ascending = True).tail(10),
             x = '% votos', 
             y = 'nome_partido',
             barmode = 'group',
             text = '% votos',
             color_discrete_sequence=['rgb(93,72,152)'],
             labels = {'nome_partido': '', '% votos': '% dos votos para deputado federal no Rio', 'ano': ''})

fig.update_yaxes(range = (-1, 10), linecolor = 'black', gridcolor = 'white', mirror = True)
fig.update_xaxes(showticklabels = False, range = (0, 4), linecolor = 'black', gridcolor = 'white', mirror = True)
fig.update_layout(width = 600, height = 400, plot_bgcolor = 'white', paper_bgcolor = 'white')
fig.update_traces(texttemplate='%{text:}%', textposition='outside', textfont = {'color': 'black', 'size': 13})
fig.show()

fig.write_image("viz/top_10.png")

In [216]:
dicionario = {
    'MDB': 'PMDB / MDB',
    'PMDB': 'PMDB / MDB',
    'eleito': 'eleito por qp',
    'AVANTE': 'PT do B / AVANTE',
    'PT do B': 'PT do B / AVANTE',
    'PRB': 'PRB / REPUBLICANOS',
    'REPUBLICANOS': 'PRB / REPUBLICANOS',
    'DEM': 'PFL / DEM',
    'PFL': 'PFL / DEM',
    'PL': 'PR / PL',
    'PR': 'PR / PL'
}

eleicoes = eleicoes.replace(dicionario)

In [217]:
flt = eleicoes['resultado'].isin(['eleito por qp', 'eleito por media'])
eleicoes = eleicoes[flt]

In [218]:
partidos = pd.DataFrame(eleicoes['sigla_partido'].value_counts().reset_index(name = 'n')).head(10)['index'].to_list()
partidos.append('UNIÃO')

In [219]:
partidos_ano = eleicoes.groupby(['ano', 'sigla_partido'])['sigla_partido'].count().reset_index(name = 'n')

flt = partidos_ano['sigla_partido'].isin(partidos)

fig = px.bar(partidos_ano[flt].sort_values(['n'], ascending = 'True'), 
             x = 'n', 
             y = 'sigla_partido', 
             facet_col = 'ano', 
             barmode = 'group',
             text = 'n',
             color_discrete_sequence=['rgb(93,72,152)'],
             category_orders = {'ano': [2010, 2014, 2018, 2022]},
             labels = {'sigla_partido': '', 'n': 'n', 'ano': ''})

fig.update_yaxes(range = (-1, 11), linecolor = 'black', gridcolor = 'lightgrey', mirror = True)
fig.update_xaxes(showticklabels = False, range = (0, 20), linecolor = 'black', gridcolor = 'white', mirror = True)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(width = 1300, height = 600, plot_bgcolor = 'white', paper_bgcolor = 'white')
fig.update_traces(texttemplate='%{text:}', textposition='outside', textfont = {'color': 'black', 'size': 13})
fig.show()

fig.write_image("viz/cadeiras_partidos_ano.png")

In [220]:
n_partidos = eleicoes.groupby(['ano'])['sigla_partido'].nunique().reset_index(name = 'n')

n_partidos['ano'] = n_partidos['ano'].astype(str)

fig = px.line(n_partidos, 
             x = 'ano', 
             y = 'n',
             text = 'n',
             color_discrete_sequence=['rgb(93,72,152)'],
             category_orders = {'ano': [2010, 2014, 2018, 2022]},
             labels = {'n': '', 'ano': ''})

fig.update_yaxes(showticklabels = False, range = (10, 25))
fig.update_layout(width = 400, height = 300, plot_bgcolor = 'white', paper_bgcolor = 'white')
fig.update_traces(texttemplate='%{text:}', textposition='top center', textfont = {'color': 'black', 'size': 13})
fig.show()

fig.write_image("viz/n_partidos_camara.png")

In [221]:
tipo_eleicao = eleicoes.groupby(['ano', 'resultado'])['resultado'].count().reset_index(name = 'n')

tipo_eleicao['resultado'] = tipo_eleicao['resultado'].str.capitalize()

fig = px.bar(tipo_eleicao.sort_values(['n', 'resultado'], ascending = 'False'), 
             x = 'n', 
             y = 'resultado', 
             facet_col = 'ano', 
             barmode = 'group',
             text = 'n',
             color_discrete_sequence=['rgb(93,72,152)'],
             category_orders = {'ano': [2010, 2014, 2018, 2022]},
             labels = {'resultado': '', 'n': 'n', 'ano': ''})

fig.update_yaxes(range = (-1, 2), linecolor = 'black', gridcolor = 'white', mirror = True)
fig.update_xaxes(showticklabels = False, range = (0, 50), linecolor = 'black', gridcolor = 'white', mirror = True)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(width = 1200, height = 300, plot_bgcolor = 'white', paper_bgcolor = 'white')
fig.update_traces(texttemplate='%{text:}', textposition='outside', textfont = {'color': 'black', 'size': 13})
fig.show()

fig.write_image("viz/tipo_eleicao.png")

In [222]:
por_genero = eleicoes.groupby(['ano', 'genero'])['genero'].count().reset_index(name = 'n')
por_genero['%'] = (por_genero['n'] / 46) * 100 

por_genero['genero'] = por_genero['genero'].str.capitalize()

fig = px.bar(por_genero.sort_values(['%', 'genero'], ascending = 'False').round(2), 
             x = '%', 
             y = 'genero', 
             facet_col = 'ano', 
             barmode = 'group',
             text = '%',
             text_auto = '.1%',
             color_discrete_sequence=['rgb(93,72,152)'],
             category_orders = {'ano': [2010, 2014, 2018, 2022]},
             labels = {'genero': '', 'n': '', 'ano': ''})

fig.update_yaxes(range = (-1, 2), linecolor = 'black', gridcolor = 'white', mirror = True)
fig.update_xaxes(showticklabels = False, range = (0, 110), linecolor = 'black', gridcolor = 'white', mirror = True)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(width = 1200, height = 300, plot_bgcolor = 'white', paper_bgcolor = 'white')
fig.update_traces(texttemplate='%{text:}', textposition='outside', textfont = {'color': 'black', 'size': 13})
fig.show()

fig.write_image("viz/genero_eleitos.png")

In [223]:
por_raca = eleicoes.groupby(['ano', 'raca'])['raca'].count().reset_index(name = 'n')
por_raca['%'] = (por_raca['n'] / 46) * 100 

por_raca['raca'] = por_raca['raca'].str.capitalize()

fig = px.bar(por_raca.sort_values(['%'], ascending = 'True').round(2), 
             x = '%', 
             y = 'raca',
             facet_col='ano',
             barmode = 'group',
             text = '%',
             text_auto = '.1%',
             color_discrete_sequence=['rgb(93,72,152)'],
             category_orders = {'ano': [2014, 2018, 2022], 'raca': {'Branca': 1, 'Parda': 2, 'Preta': 3, 'Nao informado': 4}},
             labels = {'raca': '', 'n': '', 'ano': ''})

fig.update_yaxes(range = (-1, 4), linecolor = 'black', gridcolor = 'white', mirror = True)
fig.update_xaxes(showticklabels = False, range = (0, 110), linecolor = 'black', gridcolor = 'white', mirror = True)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(width = 1200, height = 400, plot_bgcolor = 'white', paper_bgcolor = 'white')
fig.update_traces(texttemplate='%{text:}', textposition='outside', textfont = {'color': 'black', 'size': 13})
fig.show()

fig.write_image("viz/raca_eleitos.png")